In [13]:
from utils.htil_toolbox import Subject, load_files, NumpyDatasetGroupSplit, nn_eval, SimpleLinear
import torch
#import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
import math
torch.manual_seed(42)



# Pass interpolate channel through to add interpolate step into the existing pipeline
_interpolate_channels = {
    '24': ['CP3', 'P2'],
    '42': ['TP8', 'TP7', "T7"],
    '49': ['T8'],
    '11': ['F5', 'F7', 'FT7', 'FC5', 'T7'],
    '12': ['C5', 'T7', 'TP7'],
    '13': ['FC6'],
    '18': ['FC3'],
    '20': ['T7'],
    '21': ['T7', 'T8', 'P2'],
    '27': ['P10'],
    '28': ['C3'],
    '5': ['F7', 'FT7']
}

def getFileID(fileName):
   return fileName.split('Abby')[0].split('/')[-1] #specific to test set 

def getFeatures():
    control_files = load_files('data/td/', ".set")
    patient_files = load_files('data/asd/', ".set")
    control_group_ids = [getFileID(i) for i in control_files]
    patient_group_ids = [getFileID(i) for i in patient_files]
    group_ids_list = control_group_ids + patient_group_ids
    control_epochs = [Subject(file, muscle_thresh=5, interpolate=_interpolate_channels).getRegionPSDFeatures() for file in control_files]
    patient_epochs = [Subject(file, muscle_thresh=5, interpolate=_interpolate_channels).getRegionPSDFeatures() for file in patient_files]
    control_epochs_labels = [len(i) * [0] for i in control_epochs]
    experiment_epochs_labels = [len(i) * [1] for i in patient_epochs]
    all_data = control_epochs + patient_epochs
    groups_list = [[group_ids_list[i]]*len(j) for i, j in enumerate(all_data)]
    all_data_stacked = np.vstack(all_data)
    all_labels = np.hstack(control_epochs_labels + experiment_epochs_labels)
    # Create array of IDs for group split that are linked to the original file ID 
    group_array = np.hstack(groups_list)
    return all_data_stacked, all_labels, group_array

def saveData():
    x, y, group = getFeatures()
    np.save('data/dataframes/x_psd.npy', x) 
    np.save('data/dataframes/y_psd.npy', y)   
    np.save('data/dataframes/group_psd.npy', group) 

def main():
    x = np.load('data/dataframes/x_psd.npy')
    y = np.load('data/dataframes/y_psd.npy')
    group = np.load('data/dataframes/group_psd.npy')
    print(x.shape, y.shape, group.shape)
    train, test = NumpyDatasetGroupSplit(x, y, group).getSplit()
    nn_eval(SimpleLinear, train, test, 4*41, 4*41*2, 2, "cpu", batch_size=100,  print_point=1)

    #print(f'Train: {len(train)}, Test: {len(test)}')

#saveData()
main()


(4056, 4, 41) (4056,) (4056,)
x, y train:  (3168, 4, 41) (3168,)
Epoch [1/1000], Step [1/32], Loss: 0.8265
Epoch [1/1000], Step [2/32], Loss: 14.4301
Epoch [1/1000], Step [3/32], Loss: 6.4518
Epoch [1/1000], Step [4/32], Loss: 1.5949
Epoch [1/1000], Step [5/32], Loss: 4.0092
Epoch [1/1000], Step [6/32], Loss: 3.4310
Epoch [1/1000], Step [7/32], Loss: 2.4203
Epoch [1/1000], Step [8/32], Loss: 0.7431
Epoch [1/1000], Step [9/32], Loss: 2.8422
Epoch [1/1000], Step [10/32], Loss: 3.5596
Epoch [1/1000], Step [11/32], Loss: 2.1058
Epoch [1/1000], Step [12/32], Loss: 0.8814
Epoch [1/1000], Step [13/32], Loss: 2.1311
Epoch [1/1000], Step [14/32], Loss: 2.2704
Epoch [1/1000], Step [15/32], Loss: 1.7471
Epoch [1/1000], Step [16/32], Loss: 1.0533
Epoch [1/1000], Step [17/32], Loss: 0.9641
Epoch [1/1000], Step [18/32], Loss: 1.7761
Epoch [1/1000], Step [19/32], Loss: 1.0371
Epoch [1/1000], Step [20/32], Loss: 0.8078
Epoch [1/1000], Step [21/32], Loss: 1.0887
Epoch [1/1000], Step [22/32], Loss: 1.27